In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.utils import custom_object_scope
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Define your custom objects
custom_objects = {'Adam': 5e-5}

In [ ]:
# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Wrap the model loading in custom_object_scope
with custom_object_scope(custom_objects):
    loaded_model = bert_model

In [ ]:
# Load and preprocess IMDB dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data()
x_train = [tokenizer.encode(text, add_special_tokens=True, max_length=128, padding='max_length', truncation=True) for text in x_train]
x_test = [tokenizer.encode(text, add_special_tokens=True, max_length=128, padding='max_length', truncation=True) for text in x_test]

In [ ]:
# Define optimizer with custom learning rate
learning_rate = 1e-5  # Adjust the learning rate as needed
optimizer = Adam(learning_rate=learning_rate)

In [ ]:
# Compile the model
loss = SparseCategoricalCrossentropy(from_logits=True)
metric = SparseCategoricalAccuracy('accuracy')
bert_model.compile(optimizer='Adam', loss=loss, metrics=[metric])

In [ ]:
# Train the model
history = bert_model.fit(x_train, y_train, epochs=3, batch_size=32, validation_data=(x_test, y_test), callbacks=[EarlyStopping(patience=2)])

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {'(<class \'list\'> containing values of types {"<class \'int\'>"})'}), <class 'numpy.ndarray'>

In [ ]:
# Evaluate model
test_loss, test_accuracy = bert_model.evaluate(x_test)
print('Test Accuracy:', test_accuracy)